## Notebook principal

Primero se busca trabajar con 3 libros de texto en formato PDF, los cuales son:  
Codigo limpio - Robert Cecil Martin  
Ingenieria de software - Pressman  
Ingenieria de software - Sommerville

In [17]:
%pip install PyPDF2
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from os import makedirs, listdir, path, getcwd

BASE_DIR = getcwd()
DATA_DIR = path.join(BASE_DIR, "data")
PDF_DIR = path.join(DATA_DIR, "pdf")
TXT_DIR = path.join(DATA_DIR, "txt")

Una vez teniendo los libros en formato PDF, se procede a convertirlos a formato TXT, para poder trabajar con ellos.
La conversion ocurre pagina por pagina.

In [19]:
makedirs(TXT_DIR, exist_ok=True)
# obtener todos los archivos pdf dentro de PDF_DIR
pdf_files = [f for f in listdir(PDF_DIR) if f.lower().endswith('.pdf')]
for pdf in pdf_files:
    # leer el archivo pdf
    with open(path.join(PDF_DIR, pdf), 'rb') as f:
        pdf_reader = PdfReader(f)
        # leer el número de páginas en el archivo pdf
        pbar = tqdm(range(len(pdf_reader.pages)))
        # iterar sobre las páginas en el archivo pdf
        page_num = 0
        for page in pbar:
            pbar.set_description(f'Processing {pdf}')
            # obtener el texto de la página eliminando los saltos de línea
            text = pdf_reader.pages[page].extract_text().replace('\n', ' ')
            # escribir el texto en un archivo txt
            txt_file = f'{pdf[:-4]}_{page_num}.txt'
            # generar un archivo txt que contenga el texto de todos los archivos txt de un mismo pdf
            main_txt_file = f'{pdf[:-4]}.txt'
            with open(path.join(TXT_DIR, txt_file), 'w', encoding='utf-8') as f, open(path.join(TXT_DIR, main_txt_file), 'a', encoding='utf-8') as mwf:
                # formatear el texto para normalizarlo
                formatted_text = text.replace('\n', ' ').lower().replace('  ', ' ')
                f.write(formatted_text)
                mwf.write(formatted_text)
                page_num += 1

Processing codigo_limpio_robert_cecil_martin.pdf: 100%|██████████| 644/644 [00:13<00:00, 48.38it/s] 
Processing ingenieria_de_software_sommerville.PDF: 100%|██████████| 792/792 [00:05<00:00, 156.39it/s]
Processing ingenieria_de_software_pressman.PDF: 100%|██████████| 810/810 [00:09<00:00, 84.90it/s] 


Una vez teniendo todos los archivos .txt se debe obtener el dataset de STS lleno de oraciones relacionadas a la ingenieria de software y un puntaje entre 0 y 1 que valore la similitud entre las oraciones.

Se buscara hacer uso de algun modelo para extraer las oraciones del archivo .txt

In [20]:
%pip install huggingface_hub transformers datasets  

Note: you may need to restart the kernel to use updated packages.


Buscando los modelos en espanol para extraer automaticamente las oraciones de ingenieria de software encontre este modelo: [hiiamsid/sentence_similarity_spanish_es](https://huggingface.co/hiiamsid/sentence_similarity_spanish_es) el cual potencialmente sera usado para realizarle fine-tuning y asi obtener un modelo final.

Se busca obtener cada oracion del archivo .txt principal de cada libro e incluirla en un archivo .json.

Primero se limpia el texto de cada archivo .txt, se eliminan los saltos de linea, espacios en blanco, caracteres especiales, palabras que no aportan informacion, palabras repetidas.

In [21]:
import re

def remove_useless_text(text):
    # Regular expressions for detecting useless text patterns
    patterns = [
        r'>',  # Remove bullet point indicators
        r'\d+\w*',  # Remove numbers or digits combined with letters
        r'\b\w\b',  # Remove single-letter words
        r'\b\w{1,2}\b',  # Remove words with one or two characters
        r'www\.[^\s]+',  # Remove URLs
    ]

    # Apply each pattern sequentially to remove useless text
    for pattern in patterns:
        text = re.sub(pattern, '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [22]:
for filename in listdir(TXT_DIR):
    if not any(char.isdigit() for char in filename):
        # por la convencion de nombres que se tomo, estos archivos son los que contienen todo el texto de su respectivo pdf
        with open(TXT_DIR + '/' + filename, 'r', encoding='utf-8') as file:
            text = file.read()
            cleaned_text = remove_useless_text(text)
            # Do something with the cleaned text, such as write it to a new file
            with open(TXT_DIR + '/cleaned_' + filename, 'w', encoding='utf-8') as cleaned_file:
                cleaned_file.write(cleaned_text)

Ya con un archivo que contiene texto "limpio" se obtienen las oraciones de cada libro y se incluyen en un archivo .json.

In [23]:
%pip install nltk  

Note: you may need to restart the kernel to use updated packages.


In [35]:
import nltk
import json

# Descargar el tokenizador de oraciones de NLTK
nltk.download('punkt')

with open('/home/abraham/personal/sts-swe/data/txt/cleaned_codigo_limpio_robert_cecil_martin.txt', 'r') as file:
    text = file.read()

sentences = nltk.sent_tokenize(text)
meaningful_sentences = []
for sentence in sentences:
    # Si la oración tiene más de 5 palabras, la consideramos significativa
    if len(sentence.split()) > 5:
        # Decodificar la oración para reemplazar los escape sequences con los caracteres correspondientes
        sentence = sentence.encode().decode('unicode_escape')
        meaningful_sentences.append(sentence)

# Crear un objeto JSON y agregar las oraciones significativas como elementos del objeto
data = {}
data['sentences'] = meaningful_sentences

with open('meaningful_sentences.json', 'w') as file:
    json.dump(data, file, indent=4)

[nltk_data] Downloading package punkt to /home/abraham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_372642/2583404250.py:17: DeprecationWarning: invalid escape sequence '\â'
  sentence = codecs.decode(sentence, 'unicode_escape')
/tmp/ipykernel_372642/2583404250.py:17: DeprecationWarning: invalid escape sequence '\!'
  sentence = codecs.decode(sentence, 'unicode_escape')
/tmp/ipykernel_372642/2583404250.py:17: DeprecationWarning: invalid escape sequence '\|'
  sentence = codecs.decode(sentence, 'unicode_escape')


In [36]:
%pip install spacy
!python -m spacy download es_core_news_sm

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 66.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [37]:
import spacy

# Load Spanish tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("es_core_news_sm")  
# you can change "es_core_news_sm" to "es_core_news_md" if you have downloaded the medium Spanish model

# Read the text file
with open('/home/abraham/personal/sts-swe/data/txt/cleaned_codigo_limpio_robert_cecil_martin.txt', 'r') as file:
    text = file.read()

# Process whole documents
doc = nlp(text)

# Tokenize sentence
sentences = [sent.text for sent in doc.sents]


ValueError: [E088] Text of length 2010322 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [ ]:
for sentence in sentences:
    # Si la oración tiene más de 5 palabras, la consideramos significativa
    if len(sentence.split()) > 5:
        # Decodificar la oración para reemplazar los escape sequences con los caracteres correspondientes
        sentence = sentence.encode().decode('unicode_escape')
        meaningful_sentences.append(sentence)

# Crear un objeto JSON y agregar las oraciones significativas como elementos del objeto
data = {}
data['sentences'] = meaningful_sentences

with open('meaningful_sentences.json', 'w') as file:
    json.dump(data, file, indent=4)

Se opta por usar los embeddings del modelo anteriormente mencionado para asi encontrar el contexto de cada oracion.